In [1]:
!pip install bert-score

^C



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for bert-score from https://files.pythonhosted.org/packages/c6/8c/bc5457de4c004b1a623b31f7bc8d0375fb699b7d67df11879098b4b7b7c8/bert_score-0.3.13-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.0.0 from https://files.pythonhosted.org/packages/5a/6a/775b93d6888c31f1f1fc457e4f5cc89f0984412d5dcdef792b8f2aa6e812/torch-2.4.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pandas>=1.0.1 from https://files.pythonhosted.org/packages/ab/63/966db1321a0ad55df1d1fe51505d2cdae191b84c907974873817b0a6e849/pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for transformers>=3.0.0 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ----------------------------------- -- 41.0/43.7 kB 991.0 kB/s eta 0:00:01
 

In [3]:
import os
print(os.listdir(eng_annotation_path))
print(os.listdir(jap_annotation_path))

['AisazuNihaIrarenai.xml', 'AkkeraKanjinchou.xml', 'Akuhamu.xml', 'AosugiruHaru.xml', 'AppareKappore.xml', 'Arisa.xml', 'ARMS.xml', 'BakuretsuKungFuGirl.xml', 'Belmondo.xml', 'BEMADER_P.xml', 'BokuHaSitatakaKun.xml', 'BurariTessenTorimonocho.xml', 'ByebyeC-BOY.xml', 'Count3DeKimeteAgeru.xml', 'DollGun.xml', 'Donburakokko.xml', 'DualJustice.xml', 'EienNoWith.xml', 'EvaLady.xml', 'EverydayOsakanaChan.xml', 'GakuenNoise.xml', 'GarakutayaManta.xml', 'GinNoChimera.xml', 'GOOD_KISS_Ver2.xml', 'Hamlet.xml', 'HanzaiKousyouninMinegishiEitarou.xml', 'HaruichibanNoFukukoro.xml', 'HarukaRefrain.xml', 'HealingPlanet.xml', 'HeiseiJimen.xml', 'HighschoolKimengumi_vol01.xml', 'HighschoolKimengumi_vol20.xml', 'HinagikuKenzan.xml', 'HisokaReturns.xml', 'JangiriPonpon.xml', 'JijiBabaFight.xml', 'Joouari.xml', 'Jyovolley.xml', 'KarappoHighschool.xml', 'KimiHaBokuNoTaiyouDa.xml', 'KoukouNoHitotachi.xml', 'KuroidoGanka.xml', 'KyokugenCyclone.xml', 'LancelotFullThrottle.xml', 'LoveHina_vol01.xml', 'LoveHina_

In [4]:
import os
from xml.etree import ElementTree

def extract_text_from_xml(xml_path, output_dir):
    tree = ElementTree.parse(xml_path)
    root = tree.getroot()
    
    # Extract all text from the XML
    text_content = []
    for text_element in root.findall(".//text"):
        
        text_content.append(text_element.text)

    # Join all the extracted text into a single string
    all_text = "\n".join(text_content)
    
    # Determine the output file path
    base_name = os.path.basename(xml_path)
    file_name = os.path.splitext(base_name)[0] + ".txt"
    output_path = os.path.join(output_dir, file_name)
    
    # Write the extracted text to a text file
    with open(output_path, "w", encoding="utf-8") as text_file:
        text_file.write(all_text)

def process_annotation_directory(annotation_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process each XML file in the directory
    for file_name in os.listdir(annotation_dir):
        if file_name.endswith(".xml"):
            xml_path = os.path.join(annotation_dir, file_name)
            extract_text_from_xml(xml_path, output_dir)
            break 

# Paths to the directories containing the XML files
eng_annotation_path = r"C:\Users\Lenovo\Downloads\annotations"
jap_annotation_path = r"C:\Users\Lenovo\Downloads\japanese annotations"

# Output directories for extracted text files
eng_output_dir = r"C:\Users\Lenovo\Downloads\extracted_annotations"
jap_output_dir = r"C:\Users\Lenovo\Downloads\extracted_japanese_annotations"

# Process the XML files in both directories
process_annotation_directory(eng_annotation_path, eng_output_dir)
process_annotation_directory(jap_annotation_path, jap_output_dir)

print("Text extraction complete.")


Text extraction complete.


In [ ]:
for file in os.listdir(eng_annotation_path):
    extract_text_from_xml(,file)

In [7]:
from bert_score import score

In [5]:
import os
import pandas as pd
from bert_score import score

def read_texts_from_directory(directory_path):
    texts = {}
    for file_name in os.listdir(directory_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read().strip()
                texts[file_name] = text
    return texts

# Paths to the directories containing the extracted texts
eng_extracted_dir = r"C:\Users\Lenovo\Downloads\extracted_annotations"
jap_extracted_dir = r"C:\Users\Lenovo\Downloads\extracted_japanese_annotations"

# Read English translations and Japanese references
english_translations = read_texts_from_directory(eng_extracted_dir)
japanese_references = read_texts_from_directory(jap_extracted_dir)

# Ensure both directories have the same number of files for comparison
if len(english_translations) != len(japanese_references):
    print("Warning: The number of English translations and Japanese references does not match.")
else:
    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['File Name', 'Precision', 'Recall', 'F1 Score'])
    
    # Calculate BERTScore for each file
    for key in sorted(english_translations.keys()):
        # Get translation and reference for the current file
        translation = [english_translations[key]]
        reference = [japanese_references[key]]
        
        # Calculate BERTScore using a multilingual model
        P, R, F1 = score(translation, reference, model_type='bert-base-multilingual-cased')

        # Create a DataFrame row for the current result
        row = pd.DataFrame({
            'File Name': [key],
            'Precision': [P[0].item()],
            'Recall': [R[0].item()],
            'F1 Score': [F1[0].item()]
        })

        # Append the row to the results DataFrame using pd.concat
        results_df = pd.concat([results_df, row], ignore_index=True)

    # Set the index of the DataFrame to 'File Name'
    results_df.set_index('File Name', inplace=True)
    
    # Output the DataFrame
    print(results_df)

    # Optionally, save the DataFrame to a CSV file
    results_df.to_csv(r"C:\Users\Lenovo\Downloads\bert_score_results.csv")


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20256\1148479314.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, row], ignore_index=True)


                        Precision    Recall  F1 Score
File Name                                            
ARMS.txt                 0.606368  0.591816  0.599003
AisazuNihaIrarenai.txt   0.576154  0.567475  0.571782
AkkeraKanjinchou.txt     0.626000  0.627488  0.626743
Akuhamu.txt              0.611579  0.612159  0.611869
AosugiruHaru.txt         0.602449  0.582010  0.592053
...                           ...       ...       ...
YouchienBoueigumi.txt    0.592239  0.568622  0.580190
YoumaKourin.txt          0.647843  0.650403  0.649121
YukiNoFuruMachi.txt      0.606533  0.583922  0.595013
YumeNoKayoiji.txt        0.598386  0.579806  0.588950
YumeiroCooking.txt       0.564944  0.533663  0.548858

[109 rows x 3 columns]


In [6]:
import os
import pandas as pd
from bert_score import score
import torch

def read_texts_from_directory(directory_path):
    texts = {}
    for file_name in os.listdir(directory_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read().strip()
                texts[file_name] = text
    return texts

# Paths to the directories containing the extracted texts
eng_extracted_dir = r"C:\Users\Lenovo\Downloads\extracted_annotations"
jap_extracted_dir = r"C:\Users\Lenovo\Downloads\extracted_japanese_annotations"

# Read English translations and Japanese references
english_translations = read_texts_from_directory(eng_extracted_dir)
japanese_references = read_texts_from_directory(jap_extracted_dir)

# Check if a GPU is available and use it if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Ensure both directories have the same number of files for comparison
if len(english_translations) != len(japanese_references):
    print("Warning: The number of English translations and Japanese references does not match.")
else:
    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['File Name', 'Precision', 'Recall', 'F1 Score'])
    
    # Calculate BERTScore for each file
    for key in sorted(english_translations.keys()):
        # Get translation and reference for the current file
        translation = [english_translations[key]]
        reference = [japanese_references[key]]
        
        # Calculate BERTScore using a multilingual model, specifying device
        P, R, F1 = score(translation, reference, model_type='bert-base-multilingual-cased', device=device)

        # Create a DataFrame row for the current result
        row = pd.DataFrame({
            'File Name': [key],
            'Precision': [P[0].item()],
            'Recall': [R[0].item()],
            'F1 Score': [F1[0].item()]
        })

        # Append the row to the results DataFrame using pd.concat
        results_df = pd.concat([results_df, row], ignore_index=True)

    # Set the index of the DataFrame to 'File Name'
    results_df.set_index('File Name', inplace=True)
    
    # Output the DataFrame
    print(results_df)

    # Optionally, save the DataFrame to a CSV file
    results_df.to_csv(r"C:\Users\Lenovo\Downloads\bert_score_results.csv")


Using device: cpu


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

In [7]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118


^C
